In [187]:
import pandas as pd

In [188]:
def read_bid_ask_data(ask_fname : str, bid_fname : str, lowercase_columns = False, set_time_index = False):
    """Reads and combines the bid and ask csv files of duksascopy historical market data, into a single OHLCV dataframe."""
    df_ask = pd.read_csv(ask_fname, infer_datetime_format=True)
    df_bid = pd.read_csv(bid_fname, infer_datetime_format=True)

    df_avg = (df_bid[["Open", "High", "Low", "Close", "Volume"]]+ df_ask[["Open", "High", "Low", "Close", "Volume"]]) / 2.0
    df_avg = df_avg[df_avg["Volume"] > 0].reset_index()
    if ("Local time" in list(df_ask.columns)) or ("Local time" in list(df_bid.columns)):
        df_avg["time"] = df_ask["Local time"]
    df_avg["time"] = df_avg["time"].str.replace(r".\d{3} GMT[+-]\d\d\d\d", '', regex = True) ## Strip ms and GMT TZ in time column
    if "index" in list(df_avg):
        print("index column found in dataframe, so dropping them")
        df_avg.drop(labels = "index", axis = 1, inplace = True)

    if lowercase_columns:
        df_avg.columns= df_avg.columns.str.lower()
        
    if set_time_index:
        df_avg["time"] = pd.to_datetime(df_avg["time"],format='%d.%m.%Y %H:%M:%S')
        df_avg = df_avg.set_index("time")      
    return df_avg

In [189]:
### DataFrame Slicing based on nr. of rows on 1m dataframe
def slice_df_by_1m_rows(df : pd.DataFrame, nr_days_to_slice : int):
    """Slice the historical dataframe from most recent to the nr. of days specified"""
    mins_per_day = 24 * 60
    nr_days_to_slice = 365 * mins_per_day
    df = df.iloc[-nr_days_to_slice:].reset_index(drop = True)
    return df

In [190]:
gu_bid_ask_files = {
    # "Bid": "GBPUSD_Candlestick_1_M_BID_26.08.2019-26.08.2022_FilterFlats_All_EET_VMillions_Local.csv",
    # "Ask": "GBPUSD_Candlestick_1_M_ASK_26.08.2019-26.08.2022_FilterFlats_All_EET_VMillions_Local.csv",
    # "Bid": "GBPUSD_Candlestick_1_M_BID_26.08.2019-26.08.2022_FilterFlats_Disabled_EET_VMillions_Local.csv",
    # "Ask": "GBPUSD_Candlestick_1_M_ASK_26.08.2019-26.08.2022_FilterFlats_Disabled_EET_VMillions_Local.csv",
    # "Bid": "GBPUSD_Candlestick_1_M_BID_26.08.2019-26.08.2022_FilterFlats_Weekends_EET_VMillions_Local.csv",
    # "Ask": "GBPUSD_Candlestick_1_M_ASK_26.08.2019-26.08.2022_FilterFlats_Weekends_EET_VMillions_Local.csv",
    # "Bid": "GBPUSD_Candlestick_1_M_BID_26.08.2019-26.08.2022_FilterFlats_Disabled_UTC_VMillions_Local.csv",
    # "Ask": "GBPUSD_Candlestick_1_M_ASK_26.08.2019-26.08.2022_FilterFlats_Disabled_UTC_VMillions_Local.csv",
    "Bid": "GBPUSD_Candlestick_1_M_BID_26.08.2019-26.08.2022_FilterFlats_Disabled_UTC_VUnits_Local.csv",
    "Ask": "GBPUSD_Candlestick_1_M_ASK_26.08.2019-26.08.2022_FilterFlats_Disabled_UTC_VUnits_Local.csv",    
                }

In [191]:
%%time
## Write everything into one single HDF5 file indexed by keys for the various symbols
folder_path = "/Users/dilip.rajkumar/Documents/Dukascopy_Historical_Data/GU_Variants/"
ask_csv_file = folder_path + gu_bid_ask_files["Ask"]
bid_csv_file = folder_path + gu_bid_ask_files["Bid"]
print(ask_csv_file,'\n',bid_csv_file)
df_gu = read_bid_ask_data(ask_csv_file, bid_csv_file, set_time_index = True)
df_gu.to_hdf(f"/Users/dilip.rajkumar/Documents/vbtpro_tuts_private/data/GU_OHLCV_3Y_m1.h5", key='GBPUSD')

/Users/dilip.rajkumar/Documents/Dukascopy_Historical_Data/GU_Variants/GBPUSD_Candlestick_1_M_ASK_26.08.2019-26.08.2022_FilterFlats_Disabled_UTC_VUnits_Local.csv 
 /Users/dilip.rajkumar/Documents/Dukascopy_Historical_Data/GU_Variants/GBPUSD_Candlestick_1_M_BID_26.08.2019-26.08.2022_FilterFlats_Disabled_UTC_VUnits_Local.csv
index column found in dataframe, so dropping them
CPU times: user 3.7 s, sys: 275 ms, total: 3.97 s
Wall time: 3.99 s


,Open,High,Low,Close,Volume
time,,,,,
2019-08-26 00:00:00,1.226705,1.226805,1.226685,1.226805,1.425000e+07
2019-08-26 00:01:00,1.226865,1.227020,1.226855,1.227020,3.000000e+06
2019-08-26 00:02:00,1.227025,1.227025,1.227005,1.227005,2.250000e+06
2019-08-26 00:03:00,1.227010,1.227060,1.227010,1.227020,4.500000e+06
2019-08-26 00:04:00,1.227040,1.227040,1.227030,1.227030,1.500000e+06
...,...,...,...,...,...
2021-10-12 05:56:00,1.174620,1.174705,1.174520,1.174600,4.448000e+07
2021-10-12 05:57:00,1.174605,1.174630,1.174380,1.174470,2.821000e+07
2021-10-12 05:58:00,1.174460,1.174640,1.174460,1.174635,2.729500e+07


In [192]:
df_gu[df_gu.index > "2019-08-27"]

,Open,High,Low,Close,Volume
time,,,,,
2019-08-27 00:01:00,1.221810,1.221910,1.221770,1.221770,6.375000e+06
2019-08-27 00:02:00,1.221815,1.221960,1.221555,1.221765,4.200000e+06
2019-08-27 00:03:00,1.221995,1.222050,1.221995,1.222050,5.055000e+06
2019-08-27 00:04:00,1.222050,1.222690,1.222005,1.222495,5.885000e+06
2019-08-27 00:05:00,1.222430,1.222485,1.222430,1.222485,3.000000e+06
...,...,...,...,...,...
2021-10-12 05:56:00,1.174620,1.174705,1.174520,1.174600,4.448000e+07
2021-10-12 05:57:00,1.174605,1.174630,1.174380,1.174470,2.821000e+07
2021-10-12 05:58:00,1.174460,1.174640,1.174460,1.174635,2.729500e+07


In [193]:
## Specify FileNames of Bid / Ask data downloaded from DukaScopy
bid_ask_files = {
    "GBPUSD" : {"Bid": "GBPUSD_Candlestick_1_M_BID_01.12.2019-01.12.2022.csv",
                "Ask": "GBPUSD_Candlestick_1_M_ASK_01.12.2019-01.12.2022.csv"},
    "EURUSD" : {"Bid": "EURUSD_Candlestick_1_M_BID_01.12.2019-01.12.2022.csv",
                "Ask": "EURUSD_Candlestick_1_M_ASK_01.12.2019-01.12.2022.csv"},
    "AUDUSD" : {"Bid": "AUDUSD_Candlestick_1_M_BID_01.12.2019-01.12.2022.csv",
                "Ask": "AUDUSD_Candlestick_1_M_ASK_01.12.2019-01.12.2022.csv"},
    "USDCAD" : {"Bid": "USDCAD_Candlestick_1_M_BID_01.12.2019-01.12.2022.csv",
                "Ask": "USDCAD_Candlestick_1_M_ASK_01.12.2019-01.12.2022.csv"},
    "USDJPY" : {"Bid": "USDJPY_Candlestick_1_M_BID_01.12.2019-01.12.2022.csv",
                "Ask": "USDJPY_Candlestick_1_M_ASK_01.12.2019-01.12.2022.csv"}                                                                            
}

In [194]:
folder_path = "/Users/dilip.rajkumar/Documents/Dukascopy_Historical_Data/"
symbol = "EURUSD"
ask_csv_file = folder_path + bid_ask_files[symbol]["Ask"]
bid_csv_file = folder_path + bid_ask_files[symbol]["Bid"]
df = read_bid_ask_data(ask_csv_file, bid_csv_file, set_time_index = True)
df

index column found in dataframe, so dropping them


,Open,High,Low,Close,Volume
time,,,,,
2019-12-02 03:29:00,1.102380,1.102515,1.102350,1.102510,9.935
2019-12-02 03:30:00,1.102510,1.102515,1.102465,1.102475,90.055
2019-12-02 03:31:00,1.102490,1.102720,1.102460,1.102695,98.770
2019-12-02 03:32:00,1.102695,1.102695,1.102675,1.102680,20.965
2019-12-02 03:33:00,1.102685,1.102685,1.102455,1.102455,46.140
...,...,...,...,...,...
2022-11-22 07:12:00,1.050770,1.050965,1.050770,1.050955,582.305
2022-11-22 07:13:00,1.050950,1.051095,1.050945,1.051030,372.185
2022-11-22 07:14:00,1.051025,1.051220,1.050990,1.051025,446.085


In [195]:
%%time
## Write everything into one single HDF5 file indexed by keys for the various symbols
folder_path = "/Users/dilip.rajkumar/Documents/Dukascopy_Historical_Data/"
for symbol in bid_ask_files.keys():
    print('\n',symbol)
    ask_csv_file = folder_path + bid_ask_files[symbol]["Ask"]
    bid_csv_file = folder_path + bid_ask_files[symbol]["Bid"]
    print(ask_csv_file,'\n',bid_csv_file)
    # df = read_bid_ask_data(ask_csv_file, bid_csv_file, set_time_index = True)
    # df.to_hdf(f"/Users/dilip.rajkumar/Documents/vbtpro_tuts_private/data/MultiAsset_OHLCV_3Y_m1.h5", key=symbol)


 GBPUSD
/Users/dilip.rajkumar/Documents/Dukascopy_Historical_Data/GBPUSD_Candlestick_1_M_ASK_01.12.2019-01.12.2022.csv 
 /Users/dilip.rajkumar/Documents/Dukascopy_Historical_Data/GBPUSD_Candlestick_1_M_BID_01.12.2019-01.12.2022.csv

 EURUSD
/Users/dilip.rajkumar/Documents/Dukascopy_Historical_Data/EURUSD_Candlestick_1_M_ASK_01.12.2019-01.12.2022.csv 
 /Users/dilip.rajkumar/Documents/Dukascopy_Historical_Data/EURUSD_Candlestick_1_M_BID_01.12.2019-01.12.2022.csv

 AUDUSD
/Users/dilip.rajkumar/Documents/Dukascopy_Historical_Data/AUDUSD_Candlestick_1_M_ASK_01.12.2019-01.12.2022.csv 
 /Users/dilip.rajkumar/Documents/Dukascopy_Historical_Data/AUDUSD_Candlestick_1_M_BID_01.12.2019-01.12.2022.csv

 USDCAD
/Users/dilip.rajkumar/Documents/Dukascopy_Historical_Data/USDCAD_Candlestick_1_M_ASK_01.12.2019-01.12.2022.csv 
 /Users/dilip.rajkumar/Documents/Dukascopy_Historical_Data/USDCAD_Candlestick_1_M_BID_01.12.2019-01.12.2022.csv

 USDJPY
/Users/dilip.rajkumar/Documents/Dukascopy_Historical_Data/U